# Model Subclassing with Keras🏍️

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.datasets import mnist

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
print(y_train.shape)
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

(60000, 28, 28)
(60000,)


### CNN -> BatchNorm -> ReLU⏱️

In [3]:
class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size=3):
        super(CNNBlock, self).__init__()
        self.conv =  layers.Conv2D(out_channels, kernel_size, padding='same')
        self.bn = layers.BatchNormalization()
        
    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor)
        x = self.bn(x, training=training)
        x = tf.nn.relu(x)
        return x

【🧪***TEST***】

In [4]:
model = keras.Sequential(
    [
        CNNBlock(32),
        CNNBlock(64),
        CNNBlock(128),
        layers.Flatten(),
        layers.Dense(10)
    ]
)
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"]
)

model.fit(x_train, y_train, batch_size=64, epochs=3, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

Epoch 1/3
938/938 - 612s - loss: 0.6007 - accuracy: 0.9425 - 612s/epoch - 652ms/step
Epoch 2/3
938/938 - 734s - loss: 0.0956 - accuracy: 0.9816 - 734s/epoch - 782ms/step
Epoch 3/3
938/938 - 658s - loss: 0.0350 - accuracy: 0.9897 - 658s/epoch - 701ms/step
157/157 - 32s - loss: 0.0462 - accuracy: 0.9859 - 32s/epoch - 206ms/step


[0.04616112262010574, 0.9858999848365784]

### ResNet🌐

In [5]:
class ResBlock(layers.Layer):
    def __inti__(self, channels):
        super(ResBlock, self).__init__
        self.cnn1 = CNNBlock(channels[0])
        self.cnn2 = CNNBlock(channels[1])
        self.cnn3 = CNNBlock(channels[2])
        self.pooling = layers.MaxPooling2D()
        self.identity_mapping = layers.Conv2D(channels[1], 3, padding = 'same')
        
    def call(self, input_tensor, training=False):
        x = self.cnn1(input_tensor, training=False)
        x = self.cnn2(x, training=training)
        x = self.cnn3(
            x + self.identity_mapping(input_tensor), trainiing=training
        )
        return self.pooling(x)

In [6]:
class ResNet_Like(keras.Model):
    def __init__(self, num_classes=10):
        super(ResNet_Like, self).__init__
        self.block1 = ResBlock([32, 32, 64])
        self.block2 = ResBlock([128, 128, 256])
        self.block3 = ResBlock([128, 256, 512])
        self.pool = layers.GlobalAveragePooling2D()
        self.classifier = layers.Dense(num_classes)
        
    def call(self, input_sensor, training=False):
        x = self.block1(input_sensor, training=training)
        x = self.block2(x, training=training)
        x = self.block3(x, training=training)
        x = self.pool(x)
        return self.classifier(x)

【🧪***TEST***】

In [ ]:
model = ResNet_Like(num_classes=10)
print(model.summary())
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"]
)

model.fit(x_train, y_train, batch_size=64, epochs=3, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)